# Problem definition

We wish to minimize

$$ I(u,v) = \frac{\theta}{2} \int_{\omega} Q_2(\nabla_s u + \tfrac{1}{2} \nabla v \otimes \nabla v) \mathrm{d}x
   + \frac{1}{24} \int_{\omega} Q_2(\nabla^2 v - B) \mathrm{d}x, $$

with $B \in \mathbb{R}^{2 \times 2}$, e.g. the identity matrix, and $Q_2$ a quadratic form, e.g. (isotropic material):

$$ Q_2 (F) = 2 \mu | \operatorname{sym} F |^2 + \frac{2 \mu \lambda}{2 \mu + \lambda}
   \operatorname{tr}^2 F, \quad F \in \mathbb{R}^{2 \times 2}, $$

or, for a specific choice of constants, the simpler $Q_2(F) = |F|^2$.
  
We work in $P_1$ with the constraints of zero mean and zero mean antisymmetric gradient. Because we only have $C^0$ elements we set $z$ for $\nabla v$ and minimize instead

$$ J(u,z) = \frac{\theta}{2} \int_{\omega} Q_2(\nabla_s u + \tfrac{1}{2} z \otimes z) \mathrm{d}x 
          + \frac{1}{24} \int_{\omega} Q_2\nabla z - B) \mathrm{d}x 
          + \mu_\epsilon \int_{\omega} |\mathrm{curl}\ z|^{2} \mathrm{d}x, $$

then recover the vertical displacements (up to a constant) by minimizing

$$ F(p,q) = \tfrac{1}{2} || \nabla p - q ||^2 + \tfrac{1}{2} || q - z ||^2. $$

This we do by solving the linear problem $D F = 0$.

Minimization of the energy functional $J$ is done via gradient descent and a line search. In particular, at each timestep we compute $d_t w \in W $ such that for all $\tau \in W$:

$$ (d_t w, \tau)_{H^1_0 \times H^2_0} = -DJ(w_t)[\tau] $$

Note that it is essential to use the full scalar product (or the one corresponding to the seminorms? check this) or we run into issues at the boundaries (to see this start with zero displacements and integrate by parts).(Also: the proper Riesz representative will only be obtained with correct scalar product).

A decoupled gradient descent in each component does not work, probably because the functional is not separately convex (see Bartels' book, p. 110, remark (iv)).

In plane displacements and gradients of out of plane displacements form a mixed function space $U \times Z$. We also have another scalar space $V$ where the potential of the out of plane gradients lives. The model is defined and solved in `run_model()`.

# Exploring the range of $\theta$

We connect to the experiments database, then query it for the experiments we are interested in and convert the collection of objects returned into a pandas dataframe, then plot.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import os
import numpy as np
import matplotlib.pyplot as pl
from incense import ExperimentLoader
from common import gather_last_timesteps

loader = ExperimentLoader(mongo_uri='mongo:27017', db_name='lvk')

Experiments can be searched by single config key, sorted and inspected:

In [ ]:
ee = sorted(loader.find_by_config_key("theta", 11), key=lambda e: e.id)
[e.config['projection'] for e in ee if e.config['init'] == 'ani_parab']

We want to query by multiple fields, so we add this function:

In [ ]:
def find_by_config_keys(loader, _name=None, _status=None, **kwargs):
    """ Assembles a mongo query to filter results by multiple config keys simultaneously.
    If a key has value None the query does not filter by that key. """
    terms = []
    if _status:
        terms.append({"status": _status})
    if _name:
        terms.append({"experiment.name": _name})
    query = {"$and": terms}
    for k, v in kwargs.items():
        if v is not None:
            k = 'config.' + k
            terms.append({k: v})
    return loader.find(query)

We assemble a list of experiments related by initial condition, mesh type and whether projections onto constraint spaces are made or not:

In [ ]:
exp_id = "0b94093"

names = [exp_id]  # [None] for all experiments 
inits = ["ani_parab"]  #["zero", "ani_parab", "ani_compression"]  # [None] for all inits
mesh_types = ["circle"] # ["circle", "rectangle", "triangle3"]  # [None] for all mesh_types
projections = [True]  # [True, False]

In [ ]:
def average_last(ss):
    return ss[-10:].mean()

results = []
labels = []
for name in names:
    for init in inits:
        for mesh_type in mesh_types:
            for projection in projections:            
                experiments = find_by_config_keys(loader, _name=name, _status="COMPLETED",
                                                  init=init, mesh_type=mesh_type,
                                                  projection=projection)
                try:
                    df = experiments.project(on=["config.theta",
                                                 "config.mu_scale",
                                                 "config.skip",
                                                 "config.max_steps",
                                                 "start_time",
                                                 "stop_time",
                                                 {"metrics.J": average_last,
                                                 "metrics.symmetry": average_last,
                                                 "metrics.constraint": average_last,
                                                 "metrics.Kxx": average_last,
                                                 "metrics.Kxy": average_last,
                                                 "metrics.Kyy": average_last,
                                                 "metrics.alpha": len}])

                    df['duration'] = df['stop_time'] - df['start_time']
                except KeyError:
                    continue
                    
                label = "_".join((str(name),
                                  init if init else "",
                                  mesh_type if mesh_type else "",
                                  "proj" if projection else "no-proj"))
                
                unique_columns = ["config.mu_scale", "config.hmin_power", "config.mesh_m", "config.mesh_n"]
                tmp = experiments.project(on=unique_columns)
                for col in (s.split('.')[1] for s in unique_columns):
                    cnt = tmp[col].unique().shape[0]
                    if cnt > 1:
                        print("WARNING Column '%s' has multiple (%d) values in '%s'!" % (col, cnt, label))

                print("Collected %d experiments in %s" % (len(df), label))

                # HACK: there is probably a better way of counting (and I could use df.rename())
                df['steps'] = df['alpha_len']
                if df['alpha_len'].mean() < 1000:
                    df['steps'] *= df['skip']
                    
                df = df.drop(['start_time', 'stop_time', 'alpha_len', 'skip'], axis=1)
                df = df.sort_values(by=['theta'])
                
                results.append(df)
                labels.append(label)

                del experiments

## Symmetry of the minimiser

In [ ]:
m = 0   # First item to consider in all series
fs = 22  # Base font size for plots

We begin with our simple definition of symmetry which simply computes the quotient of the two principal axes of the deformed disc or the diagonals of the deformed square.

In [ ]:
pl.figure(figsize=(16, 11))
for res, label in zip(results, labels):
    n = len(res)
    #n = np.searchsorted(res['theta'], 4)
    pl.plot(res['theta'][m:n], res['symmetry_average_last'][m:n], marker='', linewidth=3, label=label)
pl.xlabel('$\\theta$', fontsize=fs)
pl.ylabel('Symmetry', fontsize=fs)
pl.tick_params(axis='both', which='major', labelsize=fs)
if len(results) == 1:
    #pl.title(labels[0], fontsize=fs-4)
    pl.savefig('theta-symmetry-%s.eps' % labels[0])
else:
    pl.hlines(1, res['theta'][m:n].min(), res['theta'][m:n].max(), colors='r', linestyles="dotted", )
    pl.legend(fontsize=fs-4)

We expect the symmetry of the solution to experience an abrupt change With increasing $\theta$, at a point where the minimiser becomes cylindrical rather than parabolic. For the zero initial condition and a circular mesh, we see indeed a very sharp increase around $\theta = 86$. Note that we use a poor criterion for symmetry (we are just taking the quotient of the principal axes), so in order to complete the picture above we plot the mean principal strains over the surface as a proxy for curvature. Ideally, we will observe a branching at the same point as above.

For other initial configurations (e.g. ani_parab) the observed behaviour is similar but the change in symmetry seems to happen gradually. One possible factor is that solutions are not necessarily minima (gradient descent might not converge to $\epsilon_{\text{stop}}$ precision in the given number of steps, see below).

In [ ]:
pl.figure(figsize=(16,11))
for res, label in zip(results, labels):
    m = 0 
    n = len(res)
    #m = np.searchsorted(res['theta'], 81)
    #n = np.searchsorted(res['theta'], 90)
    label = "" if len(results) == 1 else "-" + label
    pl.plot(res['theta'][m:n], res['Kxx_average_last'][m:n], marker='', linewidth=3, label="$K_{xx}$" + label)
    pl.plot(res['theta'][m:n], res['Kyy_average_last'][m:n], marker='', linewidth=3, label="$K_{yy}$" + label)
    
pl.xlabel('$\\theta$', fontsize=fs)
pl.ylabel('Principal strains', fontsize=fs)
pl.legend(fontsize=fs-4)
pl.tick_params(axis='both', which='major', labelsize=fs)
if len(results) == 1:
    #pl.title(labels[0], fontsize=fs-4)
    pl.savefig('theta-strains-%s.eps' % labels[0])

## The $curl$ constraint 

As an indication of the quality of the iterations, we can track how much the constraint $\mu_\epsilon \int_{\omega} |\mathrm{curl}\ z|^{2} \mathrm{d}x$ is violated. Note that with the choice $\mu_\epsilon = 1/\epsilon^\alpha$, the finer the mesh, stronger the penalty on $z$ for not being a gradient. However, we must keep $\mu_\epsilon = o(\epsilon^{-2})$ as $\epsilon \rightarrow \infty$ in order for the proof of $\Gamma$-convergence to hold.

In [ ]:
pl.figure(figsize=(12, 8))
for res, label in zip(results, labels):
    m = 0
    n = len(res)
    #m = np.searchsorted(res['theta'], 25)
    #n = np.searchsorted(res['theta'], 200)
    pl.plot(res['theta'][m:n], res['constraint_average_last'][m:n]*1e4, marker='.', label=label)
pl.hlines(0, res['theta'][m:n].min(), res['theta'][m:n].max(), colors='r', linestyles="dotted")
pl.xlabel('$\\theta$', fontsize=fs)
pl.ylabel('$|curl|*10^4$', fontsize=fs)
pl.tick_params(axis='both', which='major', labelsize=fs)
if len(results) == 1:
    pl.title(labels[0], fontsize=fs-4)
    pl.savefig('theta-curl-%s.eps' % labels[0])
else:
    pl.legend(fontsize=fs-4)

## Computational cost

As a poor-man's proxy of a proper empirical analysis of the convergence rate, we plot the duration of the experiments. This can help identify bogus runs, e.g. for having reached the maximum number of steps in too little time. Of particular interest are those where the maximum number of iterations was reached. We mark them with a cross:

In [ ]:
pl.figure(figsize=(12, 8))
minh, maxh = np.inf, 0
for res, label in zip(results, labels):
    n = len(res)
    steps = res['steps'][m:n]
    max_steps = res['max_steps'][m:n]
    seconds = res['duration'][m:n].astype('timedelta64[s]')
    minh, maxh = min(minh, int(seconds.min()//3600)), max(maxh, int(seconds.max()//3600+1))
    pl.plot(res['theta'][m:n], seconds, marker='.', label=label)
    pl.scatter(res['theta'][m:n][steps == max_steps], seconds[steps == max_steps], marker='x', s=80, label=None)
pl.xlabel('$\\theta$', fontsize=fs)

pl.ylabel('Duration (hours)', fontsize=fs)

pl.yticks(ticks=[3600*h for h in range(minh, maxh, 2)], labels=range(minh, maxh, 2))
b, t = pl.ylim()
pl.ylim(b, t+1)
pl.legend()
if len(results) == 1:
    pl.title(labels[0], fontsize=fs-4)
else:
    pl.legend(fontsize=fs-4)
pl.tick_params(axis='both', which='major', labelsize=fs)

We can also plot the number of steps directly:

In [ ]:
pl.figure(figsize=(12, 8))
for res, label in zip(results, labels):
    n = len(res)
    steps = res['steps'][m:n]
    max_steps = res['max_steps'][m:n]
    pl.plot(res['theta'][m:n], steps, marker='.', label=label)
    pl.scatter(res['theta'][m:n][steps == max_steps], steps[steps == max_steps], marker='x', s=80, label=None)
pl.xlabel('$\\theta$', fontsize=fs)
pl.ylabel('Number of steps', fontsize=fs)
if len(results) == 1:
    pl.title(labels[0], fontsize=fs-4)
else:
    pl.legend(fontsize=fs-4)
pl.tick_params(axis='both', which='major', labelsize=fs)

## Displaying the final configuration for increasing $\theta$

The function `gather_last_timesteps()` traverses all output files for an experiment and creates a ParaView file for visualisation with the last frame of each run (i.e. for all values of $\theta$. Open with ParaView to examine.

In [ ]:
gather_last_timesteps('../output', exp_id, copy_files=False)

# Some examples to query the database

Here are a few recipes to query and manipulate objects in the database.

In [ ]:
failed_exps = loader.find({"$and": [{"status": {"$ne": "COMPLETED"}}, {"experiment.name": exp_id}]})
print("Found %d failed OR NOT COMPLETED experiments with id %s" %  (len(failed_exps), exp_id))

# Careful!! `find_by_key` interprets strings as regexes!!!
#exps = loader.find_by_key("experiment.name", "^%s$" % exp_id)  #"^some name here$")
completed_exps = loader.find({"$and": [{"status": "COMPLETED"}, {"experiment.name": exp_id}]})
print("Found %d completed experiments with id %s" % (len(completed_exps), exp_id))

We can print the thetas for which the experiment failed in order to try them again. Note that if there were any output files we need to delete them or the experiment will fail since it will refuse to overwrite any files:

In [ ]:
failed_thetas = [e.config.theta for e in failed_exps]
completed_thetas = [e.config.theta for e in completed_exps]

missing = sorted(set(failed_thetas) - set(completed_thetas))

if missing:
    print("Run these in a console after triple checking!")
for m in missing:
    path = '../output/%s/0%.4f-07.70/' % (exp_id, m)
    if os.path.exists(path):
        print("rm -rf %s " % path)

Alternatively we can look for gaps in the values of theta for which experiments have completed, if we tried them with a known stride

In [ ]:
from functools import reduce

def check(l: list, step: float, acc: list) -> list:
    """ Checks for missing gaps in a sequence 
    >>> check([0, 1, 2, 3, 4, 6, 7, 8, 10, 14], 1, [])
    [5, 9, 11, 12, 13]
    """
    if len(l) > 1:
        if np.isclose(l[1], l[0] + step):
            return check(l[1:], step, acc)
        else:
            return check(l[1:], step, acc+list(np.arange(l[0]+step, l[1], step)))
    return acc

missing_for_gap = check(sorted(completed_thetas), .5, [])
print(missing_for_gap)

Deleting experiments from the database is simple:
**CAREFUL:** `failed_exps` could contain running experiments (FIXME)

In [ ]:
for e in failed_exps:
    e.delete() #confirmed=True) # to skip confirmation

We can also interact with MongoDB using the console and mongo commands. From a shell, having the containers up, run:
```shell
docker exec -it lvk_mongo_1 mongo
```

Then from the console you can access the experiments database:
```
> use lvk
> db.runs.update({"experiment.name": "ee40d72"}, {$set: {"experiment.name": "some descriptive name"}}, {multi: true})
```

Note however that manipulating runs in this manner requires manually taking care of artifacts and other related bits of information. `incense` does this for us and deletes unnecessary files and objects from the DB when we delete experiments.

Finally, [pymongo](https://api.mongodb.com/python/current/) is also installed in the container and can be used instead of the console.